In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# — Cell 3: preprocess data (with log-transform on the target) —

data = dataset.copy()

data = pd.get_dummies(data, columns=['sex', 'smoker', 'region'])

train_dataset = data.sample(frac=0.8, random_state=42)
test_dataset  = data.drop(train_dataset.index)

train_labels = train_dataset.pop('expenses')
test_labels  = test_dataset.pop('expenses')

train_labels = np.log1p(train_labels)
test_labels  = np.log1p(test_labels)

print("train data:",  train_dataset.shape)
print("train labels (log):", train_labels.shape)
print("test data:",   test_dataset.shape)
print("test labels (log):",  test_labels.shape)


In [ ]:
# — Cell 4: normalize features & override datasets —

train_stats = train_dataset.describe().transpose()

def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset).fillna(0)
normed_test_data  = norm(test_dataset).fillna(0)

train_dataset = normed_train_data
test_dataset  = normed_test_data

print("train_dataset shape:", train_dataset.shape)
print("test_dataset  shape:", test_dataset.shape)
print("Any NaNs left?:    ", train_dataset.isna().any().any())


In [ ]:
# — Cell 5: build, compile, and train a tuned regression model —

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def build_model():
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=[train_dataset.shape[1]]),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        # Output
        layers.Dense(1)
    ])
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae', 'mse']
    )
    return model

model = build_model()
model.summary()

# Callbacks:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=30,
    restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=10,
    min_lr=1e-6
)

# Train
history = model.fit(
    train_dataset,
    train_labels,
    epochs=2000,       #
    validation_split=0.2,
    verbose=0,
    callbacks=[early_stop, reduce_lr, tfdocs.modeling.EpochDots()]
)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
